# Regular Tracking + Taking Action = Positive Health Gains

### Setup

In [ ]:
import pathlib

In [ ]:
import pandas as pd

In [ ]:
date_partition = '20200922'

In [ ]:
home = pathlib.Path.home()

In [ ]:
data_input_path = f"{home}/small-data/apple-health-csv/full-extract/{date_partition}"

In [ ]:
study_path = f"{home}/small-data/study/health-stories/{date_partition}"

In [ ]:
pathlib.Path(study_path).mkdir(parents=True,exist_ok=True)

In [ ]:
bodymass = pd.read_csv(f"{data_input_path}/bodymass-summary.csv",parse_dates=['date'])

In [ ]:
body_weight = bodymass.rename(columns={'bodymass':'weight'})

In [ ]:
body_weight = body_weight.loc[body_weight['date']>'2018-04-01']

### Daily Perspective of Weight

In [ ]:
 daily_weight = body_weight.plot.line(title="Daily Weight, April 4, 2019 - September 22, 2020", 
                                      x='date', y='weight', 
                                      figsize=(14, 7), style='.-', 
                                      markevery=1, markersize=12, markerfacecolor='orange')

In [ ]:
fig = daily_weight.get_figure()

In [ ]:
fig.savefig(f"{study_path}/daily_weight.png")

### Monthly Statistics

In [ ]:
weights_by_month = body_weight.copy()
del weights_by_month['unit']

In [ ]:
weights_by_month['year_month'] = weights_by_month['date'].apply(lambda x: f"{x.year}-{x.month:02}")

In [ ]:
monthly_average_weight = weights_by_month.groupby(by="year_month").mean()

In [ ]:
monthly_average_weight = monthly_average_weight.rename(columns={'weight': 'average_weight_lbs'})

In [ ]:
maw_figure = monthly_average_weight.plot.line(title="Monthly Average Weight",
                                              figsize=(14,7), style='.-', 
                                              markerfacecolor='orange',
                                              markersize=12)

In [ ]:
maw_figure.get_figure().savefig(f"{study_path}/monthly_average_weight.png")

In [ ]:
monthly_weigh_ins = weights_by_month.groupby(by="year_month").count()

In [ ]:
monthly_weigh_ins = monthly_weigh_ins.rename(columns={"weight": "weigh_ins"})

#### Monthly and Cumulative Weight Loss/Gain

In [ ]:
monthly_first_weighin = weights_by_month.groupby(['year_month'], as_index = False).first().loc[:, ("year_month", "weight")]

In [ ]:
monthly_first_weighin['monthly_change'] = monthly_first_weighin['weight'].diff(periods=1)

In [ ]:
monthly_first_weighin['cumulative_change'] = monthly_first_weighin['monthly_change'].cumsum()

In [ ]:
mfwi_plot = monthly_first_weighin.plot.bar(x='year_month', 
                                y=['monthly_change', 'cumulative_change'], 
                                title="Monthly Weight Loss/Gain (lbs) (First Weigh-in Each Month)",
                                width=0.75,
                                figsize = (14, 7), ylim=(-35, 15))
                               

In [ ]:
monthly_weigh_ins = weights_by_month.groupby(by="year_month").count()

In [ ]:
monthly_weigh_ins = monthly_weigh_ins.rename(columns={"weight": "weigh_ins"})

In [ ]:
monthly_first_weighin.dtypes

In [ ]:
mfwi_plot.get_figure().savefig(f"{study_path}/monthly-weight-change-barplot")

In [ ]:
monthly_weigh_ins = weights_by_month.groupby(by="year_month").count()

In [ ]:
monthly_weigh_ins = monthly_weigh_ins.rename(columns={"weight": "weigh_ins"})

In [ ]:
monthly_stats = pd.merge(monthly_average_weight, monthly_weigh_ins, left_on='year_month', right_on='year_month')

In [ ]:
monthly_stats.plot.bar(y=['average_weight_lbs', 'weigh_ins'], stacked=True, 
                       figsize=(17, 7))